<a href="https://colab.research.google.com/github/jblss888/cs39aa_project/blob/main/project_part3_Juliet_Smith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Part 3, Juliet Smith


In [2]:
#Installations

!pip install datasets
!pip install nltk 
!pip install transformers

# https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 13.8 MB/s 
     |████████████████████████████████| 212 kB 55.2 MB/s 
     |████████████████████████████████| 182 kB 54.5 MB/s 
     |████████████████████████████████| 115 kB 40.3 MB/s 
     |████████████████████████████████| 127 kB 41.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 10.6 MB/s 
     |████████████████████████████████| 7.6 MB 36.1 MB/s 


In [3]:
import pandas as pd # this is the tool I will use to inteact with the dataset
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split

from datasets import load_dataset # this will let me load the dataset "LiMit" from Hugging Face

import nltk # This will help me break up sentences into individual words
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# Loaing "LiMiT" dataset from hugging face
dataset = load_dataset("limit")

Generating train split:   0%|          | 0/23559 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset limit downloaded and prepared to /root/.cache/huggingface/datasets/limit/default/1.0.0/6319c49a88c20afbe30e3479adf2b0904cfb0a8244b435d8d18f894f485636e6. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

This redoes most of what I had done from project_1. 

Just as a reminder. This dataset is already split into training and validation datasets. I will mark them as train and val. Each of them will go through the same clean up process but I will only describe the training dataset.

In [5]:
#This will convert the the hugging face dataset framework into a panda's dataset framework
#This dataset already has a set of test data so I don't need to set aside a portion of the training data. 

train = pd.DataFrame(dataset['train'])
val = pd.DataFrame(dataset['test'])
train.set_index('id')
val.set_index('id')

,sentence,motion,motion_entities
id,,,
0,"""We will go up stairs to my room,"" said his un...",yes,"[{'entity': 'paper', 'start_index': 79}, {'ent..."
1,Tom bent his steps in the direction of a large...,yes,"[{'entity': 'Tom', 'start_index': 0}]"
2,"They had drunk until their eyes had grown dim,...",yes,"[{'entity': 'They', 'start_index': 0}]"
3,He locked and bolted the door.,yes,"[{'entity': 'He', 'start_index': 0}, {'entity'..."
4,But appearing in Court in the guise of a mille...,yes,"[{'entity': 'Matthew Hale', 'start_index': 53}]"
...,...,...,...
995,We see a man in the pool with a cap on.,no,[]
996,A man is seen kneeling on a floor and puts his...,yes,"[{'entity': 'man', 'start_index': 2}, {'entity..."
997,the woman is in the hall and talk to the boys...,no,[]


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  TrainingArguments, Trainer
from datasets import Dataset, load_metric

In [7]:
MODEL_NAME = "bert-base-cased"
MAX_LENGTH=30

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
classes = train.motion.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{'yes': 0, 'no': 1}
{0: 'yes', 1: 'no'}


In [9]:
train['label'] = train['motion'].apply(lambda x: class_tok2idx[x])
val['label'] = val['motion'].apply(lambda x: class_tok2idx[x])
train.head()

,id,sentence,motion,motion_entities,label
0,0,A little boy holding a yellow ball walks by.,yes,"[{'entity': 'little boy', 'start_index': 2}, {...",0
1,1,The camel walks as the woman leans forward.,yes,"[{'entity': 'camel', 'start_index': 4}, {'enti...",0
2,2,The man mixes up various ingredients and begi...,yes,"[{'entity': 'man', 'start_index': 4}, {'entity...",0
3,3,He continues laying tiles on the floor while ...,yes,"[{'entity': 'He', 'start_index': 0}, {'entity'...",0
4,4,Then one man stands in a field holding a woode...,yes,"[{'entity': 'man', 'start_index': 9}, {'entity...",0


In [10]:
ds_raw = Dataset.from_pandas(train[['label','sentence']])
ds_raw[0]

{'label': 0, 'sentence': ' A little boy holding a yellow ball walks by.'}

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

ds = ds_raw.map(tokenize_function, batched=True)

  0%|          | 0/24 [00:00<?, ?ba/s]

In [12]:
ds[0]

{'label': 0,
 'sentence': ' A little boy holding a yellow ball walks by.',
 'input_ids': [101,
  138,
  1376,
  2298,
  2355,
  170,
  3431,
  3240,
  7104,
  1118,
  119,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [13]:
val_raw = Dataset.from_pandas(val[['label','sentence']])
val_raw[0]

{'label': 0,
 'sentence': '"We will go up stairs to my room," said his uncle, rising, and laying down the paper.'}

In [14]:
ds_train = ds
ds_eval = val_raw.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 23559
len(ds_eval) = 1000


In [16]:
import os
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=4, 
                                  do_train=True,
                                  report_to=None,
                                  output_dir='/content/',
                                  evaluation_strategy="steps",
                                  eval_steps=500,
                                  per_device_train_batch_size=10,
                                  per_device_eval_batch_size=10)

trainer = Trainer(model = model, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)

torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 23559
  Num Epochs = 4
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 9424
  Number of trainable parameters = 108312579


Step,Training Loss,Validation Loss,Accuracy
500,0.482000,0.468867,0.823000
1000,0.428000,0.414040,0.825000
1500,0.413600,0.447409,0.798000
2000,0.408900,0.431043,0.804000
2500,0.385400,0.576350,0.816000
3000,0.314300,0.468333,0.811000
3500,0.316300,0.489104,0.814000
4000,0.326700,0.481618,0.821000
4500,0.320500,0.603591,0.812000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 10
Saving model checkpoint to /content/checkpoint-500
Configuration saved in /content/checkpoint-500/config.json
Model weights saved in /content/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 10
Saving model checkpoint to /content/checkpoint-1000
Configuration saved in /content/checkpoint-1000/config.json
Model weights saved in /content/checkpo